## Objectives

This notebook contains codes to detect outliers in a given dataset of energy consumption values, using the time series decomposition method. It loops through all the accounts and labels whether the value associated to each month is an anomaly or not.
- Input: a csv file that contains the three columns needed: Account (account id), Month (calendar month), Value (numerical column to detect outliers from)
- Output: an R tibble (data frame) of the original dataset together an outlier indicator column and outlier rank column

## System Setup

This notebook includes R codes and we will need the following R packages:
- "tidyverse" package for data manipulation
- "Anomalize" package for STL time series decomposition
- "ggQC" package for drawing of XmR charts

## Load packages

In [1]:
library(tidyverse)
library(anomalize)
library(ggQC)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0     ✔ purrr   0.2.5
✔ tibble  2.0.1     ✔ dplyr   0.7.8
✔ tidyr   0.8.2     ✔ stringr 1.3.1
✔ readr   1.3.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


## Read in dataset

In [2]:
# read in the csv file which contains the prorated and imputed KWH_Consumption data for NYCHA accounts
tb_all = read_csv("../output/NYCHA_TS.csv")

# filter out unnecessary columns
tb_all = select(tb_all, c('Account', 'Month', 'Value'))

Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  X1 = col_double(),
  Account = col_character(),
  Month = col_date(format = ""),
  Value = col_double()
)


### Get the list of account_id's in the input file

In [3]:
accounts = tb_all %>% group_by(Account) 
accounts <- accounts %>% summarise(counts = n(), na_counts = sum(is.na(Value)))

# calculate the percentage of months of missing value
accounts <- mutate(accounts, na_perc = na_counts/counts)

### Fill the missing values in the "Value" column with 0

In [4]:
tb_all <- mutate(tb_all, Value = ifelse(is.na(Value), 0, Value))

## Loop through all accounts to apply STL decomposition and ourlier detection methods

### Define a function to calculate the deviation of a residual from the limit values - This will be used to rank the outliers detected based on its deviation from the boundary values

In [5]:
cal_dev <- function(residual, upper, lower) {
  if (residual < lower) {
    return(lower - residual)
  } else if (residual > upper) {
    return(residual - upper)
  } else {
    return (NA)
  }
}

### Loop through all the accounts
- apply STL decomposition to an account's time series values
- detect outliers by appling three methods applied to the residual component from the STL decomposition
- assign anomaly label and anomaly rank to each of the outlier detected from a "voted" result from the step above
    - If IQR_6X methods outputs it as an anomaly, then label is as an outlier
    - If IQR_6X methods outputs it as a normal value, but both other two methods output it as an outlier, label it as an outlier

In [6]:
results_stl <- vector("list", length(accounts$Account)) 

start.time <- Sys.time()

for (i in 1:length(accounts$Account)) {
    # select the data for the input account
    ts = select(filter(tb_all, Account == accounts$Account[[i]]), 'Month', 'Value')
    
    # apply STL decomposition
    ts_anomalized <- ts %>%
        time_decompose(Value, merge = TRUE, method = 'stl')
    
    # rename and reorder the columns of the resulting dataframe
    ts_anomalized$Account = accounts$Account[[i]]
    ts_anomalized$Missing_Value = ts_anomalized$Value == 0
    ts_anomalized <- rename(ts_anomalized, Calendar_Month = Month, Total = Value, Trend = trend, Seasonal = season
       , Residual = remainder)
    ts_anomalized <- ts_anomalized[, c('Account', 'Calendar_Month', 'Missing_Value', 'Total', 'Trend', 'Seasonal', 'Residual')]


    # Calculate residuals
    iqr_3X <- ts_anomalized %>%
            anomalize(Residual, method = 'iqr', alpha = 0.05)

    iqr_3X <- iqr_3X[, c('Account', 'Calendar_Month', 'Residual', 'Residual_l1', 'Residual_l2', 'anomaly')]
    iqr_3X <- rename(iqr_3X, Lower = Residual_l1, Upper = Residual_l2, Anomaly = anomaly)
    iqr_3X <- mutate(iqr_3X, Anomaly = ifelse(Anomaly == "Yes", TRUE, FALSE))

    iqr_6X <- ts_anomalized %>%
            anomalize(Residual, method = 'iqr', alpha = 0.025)

    iqr_6X <- iqr_6X[, c('Account', 'Calendar_Month', 'Residual', 'Residual_l1', 'Residual_l2', 'anomaly')]
    iqr_6X <- rename(iqr_6X, Lower = Residual_l1, Upper = Residual_l2, Anomaly = anomaly)
    iqr_6X <- mutate(iqr_6X, Anomaly = ifelse(Anomaly == "Yes", TRUE, FALSE))

    ctrl_limits <- QC_Lines(data = ts_anomalized$Residual, method = "XmR")  
    ctrl_limits <- ctrl_limits[, c('xBar_one_LCL', 'xBar_one_UCL')]
    ctrl_limits <- rename(ctrl_limits, Lower= xBar_one_LCL, Upper = xBar_one_UCL)
    xmr_mean <- cbind(ts_anomalized[, c('Account', 'Calendar_Month', 'Residual')], ctrl_limits)
    
    
    # Add deviation from limit, rank of outlier and outlier indicator
    iqr_3X$Dev <- mapply(cal_dev, iqr_3X$Residual, iqr_3X$Upper, iqr_3X$Lower)
    iqr_3X <- arrange(iqr_3X, desc(Dev))
    iqr_3X$Rank = seq(1:nrow(iqr_3X))
    iqr_3X[is.na(iqr_3X$Dev), ]$Rank <- NA

    iqr_6X$Dev <- mapply(cal_dev, iqr_6X$Residual, iqr_6X$Upper, iqr_6X$Lower)
    iqr_6X <- arrange(iqr_6X, desc(Dev))
    iqr_6X$Rank = seq(1:nrow(iqr_6X))
    iqr_6X[is.na(iqr_6X$Dev), ]$Rank <- NA

    xmr_mean$Dev <- mapply(cal_dev, xmr_mean$Residual, xmr_mean$Upper, xmr_mean$Lower)
    xmr_mean <- arrange(xmr_mean, desc(Dev))
    xmr_mean$Rank = seq(1:nrow(xmr_mean))
    xmr_mean[is.na(xmr_mean$Dev), ]$Rank <- NA
    xmr_mean <- mutate(xmr_mean, Anomaly = ifelse(is.na(Dev), FALSE, TRUE))
    
    
    # rename the columns
    iqr_3X <- rename(iqr_3X, Lower_3X = Lower, Upper_3X = Upper, Anomaly_3X = Anomaly, Dev_3X = Dev, Rank_3X = Rank)
    iqr_6X <- rename(iqr_6X, Lower_6X = Lower, Upper_6X = Upper, Anomaly_6X = Anomaly, Dev_6X = Dev, Rank_6X = Rank)
    xmr_mean <- rename(xmr_mean, Lower_xmr = Lower, Upper_xmr = Upper, Anomaly_xmr = Anomaly, Dev_xmr = Dev, Rank_xmr = Rank)

    
    # Combine the results of 3 methods
    result <- 
    ts_anomalized[c('Account', 'Calendar_Month', 'Missing_Value', 'Total', 'Trend', 'Seasonal', 'Residual')] %>% 
        inner_join(iqr_3X[, -3], by = c('Account', 'Calendar_Month')) %>%
        inner_join(iqr_6X[, -3], by = c('Account', 'Calendar_Month')) %>%
        inner_join(xmr_mean[, -3], by = c('Account', 'Calendar_Month')) 

    # considered as an outlier if 
    #     1) STL_6X outputs it as an outliers OR
    #     2) STL_6X outputs it as a normal point but both other two methods output it as an outlier
    result <- mutate(result
        , Anomaly_Voted = ifelse(((Anomaly_6X == TRUE) | ((Anomaly_6X == FALSE) & (Anomaly_xmr == TRUE & Anomaly_3X == TRUE))), TRUE, FALSE)
    )

    # weighted rank of the outlier
    results_stl[[i]] <- 
        mutate(result, Rank_Weighted = ifelse(is.na(Rank_6X), (Rank_3X + Rank_xmr)/2, Rank_6X))

}

end.time <- Sys.time()
time.taken.stl <- end.time - start.time

Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 12 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 41 months
Converting

frequency = 12 months
trend = 41 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 40 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 37 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 26 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency 

frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 39 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 32 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 32 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency 

frequency = 12 months
trend = 38.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 38.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 38.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 38.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 38.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 38.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 38.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 38.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 38.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index 

trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 51 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 48

...

Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 38.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 38.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 38.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 50.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Co

frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency 

Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 39 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 39 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 39 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 31.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 31.5 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 39 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 39 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 39 months
Conver

frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 43 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 43 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 43 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 43 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency 

Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 50 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 40 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 50 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 32 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 50 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 12 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 27 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 42 months
Converting from tbl_df to tbl_time.
Auto-index message: index = Month
frequency = 12 months
trend = 4 months
Converting 

## Calculate average runtime per account

In [7]:
time.taken.stl/length(accounts$Account)

Time difference of 0.00170207 mins

## Get a combined data frame of the original data with anomaly indicator and anomaly rank columns

In [8]:
result = tibble()

for (i in 1:length(accounts$Account)) {
    tmp = select(results_stl[[i]] , c('Account', 'Calendar_Month', 'Total', 'Anomaly_Voted', 'Rank_Weighted'))
    tmp = rename(tmp, Value = Total, Month = Calendar_Month, Anomaly = Anomaly_Voted, Anomaly_Rank = Rank_Weighted)
    result = rbind(result, tmp)
    }

## Save the output

The user can then output the result to any desired directory

In [9]:
# write.table(result , file = "name_of_the_directory")